In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame(
    {
        "reference" : ["ola", "uber", "lyft", "gojek", "grab"], 
        "revenue" : [1, 2, 3, 4, 5],
        "profit" : [34, 56, 32, 67, 90],
        
    }
)

df2 = pd.DataFrame(
    {
        "reference" : ["ola", "uber", "uber", "ola"], 
        "revenue" : [1, 2, 3, 4],
        "profit" : [89, 23, 54, 34]
    }
)

In [3]:
# Merging DataFrames
df3 = pd.merge(df1, df2, on = "reference", suffixes = ["_1", "_2"], how = "left") # right, inner, outer
df3

,reference,revenue_1,profit_1,revenue_2,profit_2
0,ola,1,34,1.0,89.0
1,ola,1,34,4.0,34.0
2,uber,2,56,2.0,23.0
3,uber,2,56,3.0,54.0
4,lyft,3,32,NaN,NaN
5,gojek,4,67,NaN,NaN
6,grab,5,90,NaN,NaN


In [4]:
# Concatenate DataFrame
# 1. Side-by-side
pd.concat([df1, df2], axis = 1)

,reference,revenue,profit,reference,revenue,profit
0,ola,1,34,ola,1.0,89.0
1,uber,2,56,uber,2.0,23.0
2,lyft,3,32,uber,3.0,54.0
3,gojek,4,67,ola,4.0,34.0
4,grab,5,90,NaN,NaN,NaN


In [5]:
# 2. One below the other
pd.concat([df1, df2], axis = 0, ignore_index = True)

,reference,revenue,profit
0,ola,1,34
1,uber,2,56
2,lyft,3,32
3,gojek,4,67
4,grab,5,90
5,ola,1,89
6,uber,2,23
7,uber,3,54
8,ola,4,34


In [6]:
# Replacing values from another series
s1 = pd.Series([3, 7, np.nan, 6, np.nan], index = ["A", "B", "C", "D", "E"])
s2 = pd.Series(np.arange(5), index = ["A", "B", "C", "D", "E"])
s3 = pd.Series(np.where(pd.isnull(s1), s2, s1), index = ["A", "B", "C", "D", "E"])
# Alternatively
s4 = s1.combine_first(s2)

In [7]:
# Reshaping of Datasets
df = pd.DataFrame(
    np.arange(8).reshape(2,4), 
    index = pd.Index(["Uber", "Grab"], name = "Cabs"), 
    columns = pd.Index(["c1", "c2", "c3", "c4"], name = "Attributes")
)
df

Attributes,c1,c2,c3,c4
Cabs,,,,
Uber,0,1,2,3
Grab,4,5,6,7


In [8]:
# Stacking of DataFrames
stack_df = df.stack()
stack_df

Cabs  Attributes
Uber  c1            0
      c2            1
      c3            2
      c4            3
Grab  c1            4
      c2            5
      c3            6
      c4            7
dtype: int32

In [9]:
# Unstacking of DataFrames
stack_df.unstack("Attributes")

Attributes,c1,c2,c3,c4
Cabs,,,,
Uber,0,1,2,3
Grab,4,5,6,7


In [10]:
# Unstacking of DataFrames
stack_df.unstack("Cabs")

Cabs,Uber,Grab
Attributes,,
c1,0,4
c2,1,5
c3,2,6
c4,3,7


In [11]:
# Stacking of Series
s1 = pd.Series([53, 10, 15], index = ["A", "B", "C"])
s2 = pd.Series([30, 23, 12], index = ["L", "M", "N"])
stack_series = pd.concat([s1, s2], keys = ["k1", "k2"])
stack_series

k1  A    53
    B    10
    C    15
k2  L    30
    M    23
    N    12
dtype: int64

In [12]:
# Unstacking of Series
unstack_series = stack_series.unstack()
unstack_series

,A,B,C,L,M,N
k1,53.0,10.0,15.0,NaN,NaN,NaN
k2,NaN,NaN,NaN,30.0,23.0,12.0


In [13]:
unstack_series.stack()

k1  A    53.0
    B    10.0
    C    15.0
k2  L    30.0
    M    23.0
    N    12.0
dtype: float64

In [14]:
# Pivot Tables
url = "https://en.wikipedia.org/wiki/Pivot_table"
df_list= pd.io.html.read_html(url)
df = df_list[0]
df

,Date of sale,Sales person,Item sold,Color of item,Units sold,Per unit price,Total price
0,10/01/13,Jones,Notebook,Black,8,25000,200000
1,10/02/13,Prince,Laptop,Red,4,35000,140000
2,10/03/13,George,Mouse,Red,6,850,5100
3,10/04/13,Larry,Notebook,White,10,27000,270000
4,10/05/13,Jones,Mouse,Black,4,700,2800


In [15]:
df.pivot("Sales person", "Item sold")

Date of sale                     Color of item                  \
Item sold          Laptop     Mouse  Notebook        Laptop  Mouse Notebook   
Sales person                                                                  
George                NaN  10/03/13       NaN           NaN    Red      NaN   
Jones                 NaN  10/05/13  10/01/13           NaN  Black    Black   
Larry                 NaN       NaN  10/04/13           NaN    NaN    White   
Prince           10/02/13       NaN       NaN           Red    NaN      NaN   

             Units sold                Per unit price                  \
Item sold        Laptop Mouse Notebook         Laptop  Mouse Notebook   
Sales person                                                            
George              NaN   6.0      NaN            NaN  850.0      NaN   
Jones               NaN   4.0      8.0            NaN  700.0  25000.0   
Larry               NaN   NaN     10.0            NaN    NaN  27000.0   
Prince              4.0   NaN      NaN        35000.0    NaN      NaN   

             Total price                    
Item sold         Laptop   Mouse  Notebook  
Sales person                                
George               NaN  5100.0       NaN  
Jones                NaN  2800.0  200000.0  
Larry                NaN     NaN  270000.0  
Prince          140000.0     NaN       NaN

In [16]:
# Duplicate Analysis 
df = pd.DataFrame({
    "col1" : ["Uber", "Ola", "Uber", "Uber", "Ola"],
    "col2" : [1,3,4,4,4]
})
df.duplicated()

0    False
1    False
2    False
3     True
4    False
dtype: bool

In [17]:
df.drop_duplicates()

,col1,col2
0,Uber,1
1,Ola,3
2,Uber,4
4,Ola,4


In [18]:
df.drop_duplicates(["col2"], keep = "last")

,col1,col2
0,Uber,1
1,Ola,3
4,Ola,4


In [19]:
# Replace values in a Series
s1 = pd.Series([10, 20, 30, 40, 50, 10, 20, 30, 40, 50])
s1.replace({
    10 : 80,
    20 : 90,
    30 : 100
})

0     80
1     90
2    100
3     40
4     50
5     80
6     90
7    100
8     40
9     50
dtype: int64

In [20]:
# Renaming indices in DataFrames
df = pd.DataFrame(
    np.arange(25).reshape(5,5), 
    index = ["UBER", "OLA", "GRAB", "GOJEK", "LYFT"], 
    columns = ["RE", "LG", "QE", "GR", "AG"]
)
# df.index = df.index.map(str.lower) # str.upper, str.title (Only first letter capital)
# Alternatively
df.rename(index = str.lower)

df.rename(index = { "UBER" : "LOSER", "OLA" : "GOLA" }, columns = { "RE" : "LOL" }, inplace = True)

In [28]:
# Binning
primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
limits = [0, 10, 20, 30, 40, 50]
bins = pd.cut(primes, limits)
bins.value_counts()

(0, 10]     4
(10, 20]    4
(20, 30]    2
(30, 40]    2
(40, 50]    3
dtype: int64

In [44]:
bins = pd.cut(primes, 5)
bins.value_counts()

(1.955, 11.0]    5
(11.0, 20.0]     3
(20.0, 29.0]     2
(29.0, 38.0]     2
(38.0, 47.0]     3
dtype: int64